In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
%matplotlib inline

In [0]:
df = pd.read_csv('/content/cal_housing_clean.csv')
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [0]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 7 columns):
housingMedianAge    20640 non-null float64
totalRooms          20640 non-null float64
totalBedrooms       20640 non-null float64
population          20640 non-null float64
households          20640 non-null float64
medianIncome        20640 non-null float64
medianHouseValue    20640 non-null float64
dtypes: float64(7)
memory usage: 1.1 MB


In [0]:
df.isnull().sum()

housingMedianAge    0
totalRooms          0
totalBedrooms       0
population          0
households          0
medianIncome        0
medianHouseValue    0
dtype: int64

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = df.drop('medianHouseValue',axis=1)
y = df['medianHouseValue']

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
minmax = MinMaxScaler()

In [0]:
minmax.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [0]:
X_train  = pd.DataFrame(data = minmax.transform(X_train),
                        columns = X_train.columns,
                        index = X_train.index)

In [0]:
X_test  = pd.DataFrame(data = minmax.transform(X_test),
                        columns = X_test.columns,
                        index = X_test.index)

In [0]:
housingMedian = tf.feature_column.numeric_column('housingMedianAge')
housingRooms = tf.feature_column.numeric_column('totalRooms')
housingTotalBedrooms = tf.feature_column.numeric_column('totalBedrooms')
housingPopulation = tf.feature_column.numeric_column('population')
housingHouseholds = tf.feature_column.numeric_column('households')
housingMedianIncome = tf.feature_column.numeric_column('medianIncome')


In [0]:
feature_cols = [housingMedian,housingRooms,housingTotalBedrooms,housingPopulation,housingHouseholds,
                housingMedianIncome]

In [0]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=20,num_epochs=2000,shuffle=True)

In [0]:
model =  tf.estimator.LinearRegressor(feature_columns=feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_zf69j3q', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fde13b9abe0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
model.train(input_fn=input_func,steps=20000)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp_zf69j3q/model.ckpt.
INFO:tensorflow:loss = 1604712500000.0, step = 1
INFO:tensorflow:global_step/sec: 389.891
INFO:tensorflow:loss = 812390350000.0, step = 101 (0.259 sec)
INFO:tensorflow:global_step/sec: 473.435
INFO:tensorflow:loss = 822260400000.0, step = 201 (0.213 sec)
INFO:tensorflow:global_step/sec: 433.66
INFO:tensorflow:loss = 1256298600000.0, step = 301 (0.231 sec)
INFO:tensorflow:global_step/sec: 596.549
INFO:tensorflow:loss = 1364827400000.0, step = 401 (0.172 sec)
INFO:tensorflow:global_step/sec: 480.963
INFO:tensorflow:loss = 1387134600000.0, step = 501 (0.209 sec)
INFO:tensorflow:global_step/sec: 458.949
INFO:ten

In [0]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=20,num_epochs=1,shuffle=True)

In [0]:
prediction = list(model.predict(predict_input_func))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp_zf69j3q/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
final_predictions = []

for array in prediction:
  final_predictions.append(array['predictions'][0])

In [0]:
final_predictions[:10]

[113.1357,
 99.58682,
 121.876,
 83.7162,
 127.42233,
 81.10673,
 116.01642,
 113.78102,
 130.12347,
 94.936134]

In [0]:
from sklearn.metrics import mean_squared_error

In [0]:
print("RMSE is :",mean_squared_error(y_test,final_predictions)**0.5)

RMSE is : 237127.7247580254
